In [1]:
import numpy as np

In [2]:
class RecurrentNeuralNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.001, time_steps=5):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lr = lr
        self.time_steps = time_steps
        self.Wxh = np.random.randn(hidden_dim, input_dim) * 0.01
        self.Whh = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.Why = np.random.randn(output_dim, hidden_dim) * 0.01
        self.bh = np.zeros((hidden_dim, 1))
        self.by = np.zeros((output_dim, 1))

    def forward(self, inputs):
        """Forward pass through time."""
        h = np.zeros((self.hidden_dim, 1))
        self.h_states = {-1: h}
        self.outputs = {}
        for t in range(self.time_steps):
            h = np.tanh(
                np.dot(self.Wxh, inputs[t])
                + np.dot(self.Whh, self.h_states[t - 1])
                + self.bh
            )
            y = np.dot(self.Why, h) + self.by
            self.h_states[t] = h
            self.outputs[t] = y
        return self.outputs

    def backward(self, inputs, targets):
        """Backward pass using Backpropagation Through Time (BPTT)."""
        dWxh = np.zeros_like(self.Wxh)
        dWhh = np.zeros_like(self.Whh)
        dWhy = np.zeros_like(self.Why)
        dbh = np.zeros_like(self.bh)
        dby = np.zeros_like(self.by)
        dh_next = np.zeros((self.hidden_dim, 1))
        for t in reversed(range(self.time_steps)):
            dy = self.outputs[t] - targets[t]
            dWhy += np.dot(dy, self.h_states[t].T)
            dby += dy
            dh = np.dot(self.Why.T, dy) + dh_next
            dh_raw = (1 - self.h_states[t] ** 2) * dh  # Derivative of tanh
            dbh += dh_raw
            dWxh += np.dot(dh_raw, inputs[t].T)
            dWhh += np.dot(dh_raw, self.h_states[t - 1].T)
            dh_next = np.dot(self.Whh.T, dh_raw)
        for param, dparam in zip(
            [self.Wxh, self.Whh, self.Why, self.bh, self.by],
            [dWxh, dWhh, dWhy, dbh, dby],
        ):
            np.clip(dparam, -5, 5, out=dparam)
            param -= self.lr * dparam

    def train(self, data, labels, epochs=100):
        """Train the network over multiple epochs."""
        for ep in range(epochs):
            loss = 0
            for i, t in zip(data, labels):
                outputs = self.forward(i)
                self.backward(i, t)
                loss += (
                    np.sum((outputs[self.time_steps - 1] - t[self.time_steps - 1]) ** 2)
                    / 2
                )
            if ep % 10 == 0:
                print(f"Epoch {ep}, Loss: {loss:.4f}")

In [3]:
time_steps = 5
input_dim = 3
hidden_dim = 4
output_dim = 2
data = [np.random.randn(time_steps, input_dim, 1) * 0.1 for _ in range(100)]
labels = [np.random.randn(time_steps, output_dim, 1) * 0.1 for _ in range(100)]
rnn = RecurrentNeuralNetwork(input_dim, hidden_dim, output_dim)
rnn.train(data, labels, epochs=50)

Epoch 0, Loss: 1.0231
Epoch 10, Loss: 1.0217
Epoch 20, Loss: 1.0217
Epoch 30, Loss: 1.0217
Epoch 40, Loss: 1.0217
